In [1]:
import json
import os
import parser
from src import dataio
import glob
from sklearn.metrics import accuracy_score
from seqeval.metrics import f1_score, precision_score, recall_score
import random

import torch
torch.backends.cudnn.benchmark = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if device != "cpu":
    torch.cuda.set_device(0)

Using TensorFlow backend.


### Korean FrameNet ###
	# contact: hahmyg@kaist, hahmyg@gmail.com #


	###DEVICE: cuda:0


In [2]:
try:
    dir_path = os.path.dirname(os.path.abspath( __file__ ))
except:
    dir_path = '.'

In [3]:
# 실행시간 측정 함수
import time

_start_time = time.time()

def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60)
    
    result = '{}hour:{}min:{}sec'.format(t_hour,t_min,t_sec)
    return result

In [4]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

with open('./data/frame_coreFE_list.json','r') as f:
    frame_coreFE = json.load(f)

def weighting(frame, args):
    weighted_args = []
    for arg in args:
        weighted_args.append(arg)
        if arg in frame_coreFE[frame]:
            weighted_args.append(arg)
        else:
            pass
    return weighted_args

In [48]:
def test(srl=False, masking=False, viterbi=False, language=False, model_path=False, 
         result_dir=False, train_lang=False, tgt=False,
         pretrained="bert-base-multilingual-cased"):
    if not result_dir:
        result_dir = '/disk/data/models/'+model_dir.split('/')[-2]+'-result/'
    else:
        pass
    if result_dir[-1] != '/':
        result_dir = result_dir+'/'
        
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
        
    if not train_lang:
        train_lang = language
    
    fname = fname = result_dir+train_lang+'_for_'+language
        
    if masking:
        fname = fname + '_with_masking_result.txt'
    else:
        fname = fname +'_result.txt'
        
    print('### Your result would be saved to:', fname)
        
    trn, dev, tst = dataio.load_data(srl=srl, language=language, exem=False)
    
    print('### EVALUATION')
    print('MODE:', srl)
    print('target LANGUAGE:', language)
    print('trained LANGUAGE:', train_lang)
    print('Viterbi:', viterbi)
    print('masking:', masking)
    print('using TGT token:', tgt)
    tic()    
        
    models = glob.glob(model_path+'*/')
    
    models = []
    
    # en_exemplar best
#     models.append('/disk/data/models/dict_framenet/enModel-with-exemplar/9/')
#     models.append('/disk/data/models/frameBERT/frameBERT_en/')
    
#     # ko best
#     models.append('/disk/data/models/framenet/koModel/35/')
    
    # mul best
#     models.append('/disk/data/models/framenet_old/mulModel-100/39/')
#     models.append('/disk/data/models/dict_framenet/mulModel-100/39/')

#     mul best
#     models.append('/disk/data/models/framenet_old/mulModel-100/39/')
#     models.append(model_path+'36/')
    models.append(model_path+'35/')
    
    eval_result = []
    for m in models:
#         m = '/disk/data/models/framenet/enModel-with-exemplar/epoch-8-joint.pt'
        print('### model dir:', m)
        print('### TARGET LANGUAGE:', language)
        torch.cuda.set_device(device)
        model = parser.ShallowSemanticParser(srl=srl,gold_pred=True, model_path=m, viterbi=viterbi, 
                                             masking=masking, language='multilingual', tgt=tgt,
                                             pretrained=pretrained)

        gold_senses, pred_senses, gold_args, pred_args = [],[],[],[]        
        gold_full_all, pred_full_all = [],[]

        for instance in tst:
            torch.cuda.set_device(device)
#             try:
            result = model.parser(instance)

            gold_sense = [i for i in instance[2] if i != '_'][0]
            pred_sense = [i for i in result[0][2] if i != '_'][0]


            gold_arg = [i for i in instance[3] if i != 'X']
            pred_arg = [i for i in result[0][3]]

            gold_senses.append(gold_sense)
            pred_senses.append(pred_sense)
            
            gold_args.append(gold_arg)
            pred_args.append(pred_arg)

            if srl == 'framenet':
                gold_full = []
                gold_full.append(gold_sense)
                gold_full.append(gold_sense)
                weighted_gold_args = weighting(gold_sense, gold_arg)
                gold_full += weighted_gold_args

                pred_full = []
                pred_full.append(pred_sense)
                pred_full.append(pred_sense)
                weighted_pred_args = weighting(pred_sense, pred_arg)
                pred_full += weighted_pred_args

                gold_full_all.append(gold_full)
                pred_full_all.append(pred_full)
                


                
                    
#             except KeyboardInterrupt:
#                 raise
#             except:
#                 print("cuda error")
#                 pass
            
#             break
            
        acc = accuracy_score(gold_senses, pred_senses)
        arg_f1 = f1_score(gold_args, pred_args)
        arg_precision = precision_score(gold_args, pred_args)
        arg_recall = recall_score(gold_args, pred_args)
        

#         epoch = m.split('/')[-1].split('-')[1]
        epoch = m.split('/')[-2]
        print('# EPOCH:', epoch)
        print("SenseId Accuracy: {}".format(acc))
        print("ArgId Precision: {}".format(arg_precision))
        print("ArgId Recall: {}".format(arg_recall))
        print("ArgId F1: {}".format(arg_f1))
        if srl == 'framenet':
            full_f1 = f1_score(gold_full_all, pred_full_all)
            full_precision = precision_score(gold_full_all, pred_full_all)
            full_recall = recall_score(gold_full_all, pred_full_all)
            print("full-structure Precision: {}".format(full_precision))
            print("full-structure Recall: {}".format(full_recall))
            print("full-structure F1: {}".format(full_f1))
        print('-----processing time:', tac())
        print('')


        model_result = []
        model_result.append(epoch)
        model_result.append(acc)
        model_result.append(arg_precision)
        model_result.append(arg_recall)
        model_result.append(arg_f1)
        if srl == 'framenet':
            model_result.append(full_precision)
            model_result.append(full_recall)
            model_result.append(full_f1)
        model_result = [str(i) for i in model_result]
        eval_result.append(model_result)
    
    
    with open(fname,'w') as f:
        if srl == 'framenet':
            f.write('epoch'+'\t''SenseID'+'\t'+'Arg_P'+'\t'+'Arg_R'+'\t'+'ArgF1'+'\t'+'full_P'+'\t'+'full_R'+'\t'+'full_F1'+'\n')
        else:
            f.write('epoch'+'\t''SenseID'+'\t'+'Arg_P'+'\t'+'Arg_R'+'\t'+'ArgF1'+'\n')
        for i in eval_result:
            line = '\t'.join(i)
            f.write(line+'\n')
            
        print('\n\t### Your result is saved at:', fname)

# eval for 25%

In [6]:
srl = 'framenet'
language = 'ko'

In [7]:
# print('\t###eval for ko Model (masking)')
# model_path = '/disk/data/models/framenet/koModel-25/'
# result_dir = '/disk/data/models/eval_result-25'

# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [8]:
# print('\t###eval for proto-distill (masking)')
# model_path = '/disk/data/models/framenet/proto_distilling-25/'
# result_dir = '/disk/data/models/eval_result-25'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='proto_distilling', 
#      model_path=model_path, result_dir=result_dir)

In [9]:
# print('\t###multilingual-for-ko (masking)')
# model_path = '/disk/data/models/framenet/mulModel-25/'
# result_dir = '/disk/data/models/eval_result-25'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='mul', 
#      model_path=model_path, result_dir=result_dir)

In [10]:
# print('\t###eval for ko Model (no masking)')
# model_path = '/disk/data/models/framenet/koModel-25/'
# result_dir = '/disk/data/models/eval_result-25'

# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [11]:
print('\t###eval for proto-distill (no masking)')
model_path = '/disk/data/models/framenet/proto_distilling-25/'
result_dir = '/disk/data/models/eval_result-25'
test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='proto_distilling', 
     model_path=model_path, result_dir=result_dir)

	###eval for proto-distill (no masking)
### Your result would be saved to: /disk/data/models/eval_result-25/proto_distilling_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.5175593486364528
ArgId Precision: 0.2794674624124871
ArgId Recall: 0.3012495360633428
ArgId F1: 0.28994998809240297
full-structure Precision: 0.37441624365482234
full-structure Recall: 0.3914520237758279
full-structure F1: 0.38274466392223333
-----processing time: 0hour:19min:57sec


	### Your result is saved at: /disk/data/models/eval_result-25/proto_distilling_for_ko_result.txt


In [12]:
print('\t###multilingual-for-ko (no masking)')
model_path = '/disk/data/models/framenet/mulModel-25/'
result_dir = '/disk/data/models/eval_result-25'
test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='mul', 
     model_path=model_path, result_dir=result_dir)

	###multilingual-for-ko (no masking)
### Your result would be saved to: /disk/data/models/eval_result-25/mul_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.5057877182656465
ArgId Precision: 0.28733614232209737
ArgId Recall: 0.30372386490164544
ArgId F1: 0.2953028207132976
full-structure Precision: 0.3748550378606999
full-structure Recall: 0.3888338522502123
full-structure F1: 0.3817165086311694
-----processing time: 0hour:20min:6sec


	### Your result is saved at: /disk/data/models/eval_result-25/mul_for_ko_result.txt


# eval for 100 (ko)

In [13]:
srl = 'framenet'

In [14]:
# srl = 'framenet'
# language = 'ko'
# print('\t###eval for ko Model (masking)')
# model_path = '/disk/data/models/framenet/koModel/'
# result_dir = '/disk/data/models/eval_result-100'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [ ]:
print('\t###eval for proto-distill (masking)')
language = 'ko'
model_path = '/disk/data/models/framenet/proto_distilling/'
result_dir = '/disk/data/models/eval_result-100'
test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='proto_distilling', 
     model_path=model_path, result_dir=result_dir)

	###eval for proto-distill (masking)
### Your result would be saved to: /disk/data/models/eval_result-100/proto_distilling_for_ko_with_masking_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


In [ ]:
print('\t###multilingual-for-ko (masking)')
language = 'ko'
model_path = '/disk/data/models/dict_framenet/mulModel-100/'
result_dir = '/disk/data/models/eval_result-100'
test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='mul', 
     model_path=model_path, result_dir=result_dir)

In [43]:
print('\t###eval for ko Model (no masking)')
model_path = '/disk/data/models/framenet/koModel/'
result_dir = '/disk/data/models/eval_result-100'
test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='ko', 
     model_path=model_path, result_dir=result_dir)

	###eval for ko Model (no masking)
### Your result would be saved to: /disk/data/models/eval_result-100/ko_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.6960957425936826
ArgId Precision: 0.4214622641509434
ArgId Recall: 0.44216256340467647
ArgId F1: 0.43156433013343
full-structure Precision: 0.5331867527827402
full-structure Recall: 0.5491084064534391
full-structure F1: 0.5410304678240258
-----processing time: 0hour:20min:24sec


	### Your result is saved at: /disk/data/models/eval_result-100/ko_for_ko_result.txt


In [44]:
print('\t###eval for proto-distill (no masking)')
model_path = '/disk/data/models/framenet/proto_distilling/'
result_dir = '/disk/data/models/eval_result-100'
test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='proto_distilling', 
     model_path=model_path, result_dir=result_dir)

	###eval for proto-distill (no masking)
### Your result would be saved to: /disk/data/models/eval_result-100/proto_distilling_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.6680400235432608
ArgId Precision: 0.42099447513812155
ArgId Recall: 0.4242236793269826
ArgId F1: 0.4226029085531181
full-structure Precision: 0.5234852208102225
full-structure Recall: 0.5275969431078403
full-structure F1: 0.5255330396475771
-----processing time: 0hour:20min:22sec


	### Your result is saved at: /disk/data/models/eval_result-100/proto_distilling_for_ko_result.txt


In [45]:
print('\t###multilingual-for-ko (no masking)')
# model_path = '/disk/data/models/framenet/mulModel-100/'
model_path = '/disk/data/models/framenet/mulModel-100/'
result_dir = '/disk/data/models/eval_result-100'
test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='mul', 
     model_path=model_path, result_dir=result_dir)

	###multilingual-for-ko (no masking)
### Your result would be saved to: /disk/data/models/eval_result-100/mul_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.6807926231116342
ArgId Precision: 0.42610866722149565
ArgId Recall: 0.4433997278238278
ArgId F1: 0.43458227234145747
full-structure Precision: 0.5309734513274337
full-structure Recall: 0.5434474950467025
full-structure F1: 0.5371380612673101
-----processing time: 0hour:20min:25sec


	### Your result is saved at: /disk/data/models/eval_result-100/mul_for_ko_result.txt


## eval for 10%

In [20]:
# print('\t###eval for ko Model (masking)')
# language = 'ko'
# model_path = '/disk/data/models/framenet/koModel-10/'
# result_dir = '/disk/data/models/eval_result-10'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [21]:
# print('\t###eval for proto-distill (masking)')
# language = 'ko'
# model_path = '/disk/data/models/framenet/proto_distilling-10/'
# result_dir = '/disk/data/models/eval_result-10'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='proto_distilling', 
#      model_path=model_path, result_dir=result_dir)

In [47]:
print('\t###multilingual-for-ko (masking)')
language = 'ko'
model_path = '/disk/data/models/framenet/mulModel-10/'
result_dir = '/disk/data/models/eval_result-10'
test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='mul', 
     model_path=model_path, result_dir=result_dir)

	###multilingual-for-ko (masking)
### Your result would be saved to: /disk/data/models/eval_result-10/mul_for_ko_with_masking_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


KeyboardInterrupt: 

In [23]:
# print('\t###eval for ko Model (no masking)')
# language = 'ko'
# model_path = '/disk/data/models/framenet/koModel-10/'
# result_dir = '/disk/data/models/eval_result-10'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [41]:
# print('\t###eval for proto-distill (no masking)')
# language = 'ko'
# model_path = '/disk/data/models/framenet/proto_distilling-10/'
# result_dir = '/disk/data/models/eval_result-10'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='proto_distilling', 
#      model_path=model_path, result_dir=result_dir)

	###eval for proto-distill (no masking)
### Your result would be saved to: /disk/data/models/eval_result-10/proto_distilling_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.4251520502256229
ArgId Precision: 0.2394453386158227
ArgId Recall: 0.23926759866386244
ArgId F1: 0.23935643564356437
full-structure Precision: 0.31554327025303
full-structure Recall: 0.31502971978488536
full-structure F1: 0.3152862858963918
-----processing time: 0hour:20min:8sec


	### Your result is saved at: /disk/data/models/eval_result-10/proto_distilling_for_ko_result.txt


In [42]:
# print('\t###multilingual-for-ko (no masking)')
# language = 'ko'
# model_path = '/disk/data/models/framenet/mulModel-10/'
# result_dir = '/disk/data/models/eval_result-10'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='mul', 
#      model_path=model_path, result_dir=result_dir)

	###multilingual-for-ko (no masking)
### Your result would be saved to: /disk/data/models/eval_result-10/mul_for_ko_result.txt

### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097
# of instances in trn: 17838
# of instances in dev: 2548
# of instances in tst: 5097
data example: [['태풍', 'Hugo가', '남긴', '피해들과', '회사', '내', '몇몇', '주요', '부서들의', '저조한', '실적들을', '반영하여,', 'Aetna', 'Life', 'and', 'Casualty', 'Co.의', '3분기', '<tgt>', '순이익이', '</tgt>', '182.6', '백만', '달러', '또는', '주당', '1.63', '달러로', '22', '%', '하락하였다.'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '이익.n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'Earnings_and_losses', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

../kaiser/src/utils.py:309: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_logits = sm(masked_logit).view(1,-1)


# EPOCH: 36
SenseId Accuracy: 0.42475966254659603
ArgId Precision: 0.23360254527655408
ArgId Recall: 0.23617468761598417
ArgId F1: 0.23488157490003075
full-structure Precision: 0.30922710725287034
full-structure Recall: 0.31255307104443814
full-structure F1: 0.3108811936936937
-----processing time: 0hour:20min:12sec


	### Your result is saved at: /disk/data/models/eval_result-10/mul_for_ko_result.txt


# eval for proto-distill (ko)

In [26]:
# print('\t###eval for proto-distill (masking)')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/proto_distilling/'

# result_dir = '/disk/data/models/proto_distilling/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='prot-_distilling', 
#      model_path=model_path, result_dir=result_dir)

In [27]:
# print('\t###multilingual-for-ko-no-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/mulModel-100/'

# result_dir = '/disk/data/models/eval_result/'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='mul', 
#      model_path=model_path, result_dir=result_dir)

In [28]:
# print('\t###eval for proto-distill (masking)')
# srl = 'framenet'
# language = 'en'
# model_path = '/disk/data/models/framenet/proto_distilling/'

# result_dir = '/disk/data/models/proto_distilling/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='prot-_distilling', 
#      model_path=model_path, result_dir=result_dir)

# eval for en for en

In [29]:
# print('\t###multilingual-for-en-masking')
# srl = 'framenet'
# language = 'en'
# model_path = '/disk/data/models/framenet/enModel-with-exemplar/'

# result_dir = '/disk/data/models/eval_result/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='en_with_exem', 
#      model_path=model_path, result_dir=result_dir)

# eval for ko for ko

In [30]:
# print('\t###multilingual-for-en-masking')
# srl = 'framenet'
# language = 'en'
# model_path = '/disk/data/models/framenet/enModel-with-exemplar/'

# result_dir = '/disk/data/models/results/framenet/enModel-with-exemplar/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='en_with_exem', 
#      model_path=model_path, result_dir=result_dir)

In [31]:
# print('\t###multilingual-for-ko-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/koModel/'
# result_dir = '/disk/data/models/results/framenet/koModel/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [32]:
# print('\t###multilingual-for-en-without-masking')
# srl = 'framenet'
# language = 'en'
# model_path = '/disk/data/models/framenet/enModel-with-exemplar/'
# result_dir = '/disk/data/models/results/framenet/enModel-with-exemplar/'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='en_with_exem', 
#      model_path=model_path, result_dir=result_dir)

In [33]:
# print('\t###ko-for-ko-without-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/koModel/'
# result_dir = '/disk/data/models/results/framenet/koModel/'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='ko', 
#      model_path=model_path, result_dir=result_dir)

In [34]:
# print('\t###en-for-ko-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = ''
# result_dir = '/disk/data/models/results/framenet/enModel-with-exemplar/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='en_with_exem', 
#      model_path=model_path, result_dir=result_dir)

In [35]:
# print('\t###en-for-ko-without-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = ''
# result_dir = '/disk/data/models/results/framenet/enModel-with-exemplar/'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='en_with_exem', 
#      model_path=model_path, result_dir=result_dir)

# eval for KFN

In [36]:
# print('\t###multilingual-for-ko-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/mulModel-100/'

# result_dir = '/disk/data/models/eval_result/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='mul', 
#      model_path=model_path, result_dir=result_dir)

In [37]:
# print('\t###multilingual-for-ko-without-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/mulModel-100/'
# model_path = '/disk/data/models/framenet/mulModel-100/'

# result_dir = '/disk/data/models/results/framenet/mulModel-100/'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='en_ko', 
#      model_path=model_path, result_dir=result_dir)

# eval for En again using mulModel

In [38]:
# print('\t###multilingual-for-en-masking')
# srl = 'framenet'
# language = 'en'
# model_path = '/disk/data/models/framenet/mulModel-100/'

# result_dir = '/disk/data/models/eval_result/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='mul', 
#      model_path=model_path, result_dir=result_dir)

In [39]:
# print('\t###multilingual-for-en-without-masking')
# srl = 'framenet'
# language = 'en'
# model_path = '/disk/data/models/framenet/mulModel-100/'
# model_path = '/disk/data/models/framenet/mulModel-100/'

# result_dir = '/disk/data/models/results/framenet/mulModel-100-for-en/'
# test(srl=srl, language=language, masking=False, viterbi=False, tgt=True, train_lang='en_ko', 
#      model_path=model_path, result_dir=result_dir)

# eval for distilling

In [40]:
# print('\t###multilingual-for-ko-masking')
# srl = 'framenet'
# language = 'ko'
# model_path = '/disk/data/models/framenet/distilling/'

# result_dir = '/disk/data/models/distilling/'
# test(srl=srl, language=language, masking=True, viterbi=False, tgt=True, train_lang='distilling', 
#      model_path=model_path, result_dir=result_dir)